## Notebook for scraping wiki page for toronto data

In this notebook, I'm scraping the Toronto Postal Codes from a wiki page for a Coursera course (see link below). The following tasks are done:

- read the wiki page and load it into a BeautifulSoup object
- parse the BS object and extract the data from the html table while writing it to a first dataframe
- then iterate over that df and copy the data while grouping the neighborhoods over the boroughs

ref.: https://www.coursera.org/learn/applied-data-science-capstone


Now import the libraries needed

In [4]:
##
## import libraries
##
import pandas as pd
import requests
from bs4 import BeautifulSoup
print('libaries imported')

libaries imported


In [5]:
def concatenate_neighbourhood(x):
    cadena = ""
    for i in range(len(x)-1):
        cadena = cadena + x.iloc[i] + ", "
    cadena += x.iloc[-1]
    return cadena

def select_Borough(x):
    ref = x.iloc[0]
    for i in range(1, len(x)):
        if ref != x.iloc[i]:
            for i in x:
                print(x)
            raise Exception("Postcode comprises two Boroughs")
    return ref

postalData = pd.DataFrame(columns = ["PostalCode", "Borough", "Neighbourhood"])

obj  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

page = obj.text

soup = BeautifulSoup(page, "html5lib")

#print(soup)

dataTable = soup.find('table', class_='wikitable sortable')
row = dataTable.find('tr')
rowData = row.find_all('th')

for row in dataTable.find_all('tr')[1:]:
    temp = []
    for cell in row.find_all('td'):
        temp.append(cell.text)
        #print(cell.text)
    print(temp)
    postalData = postalData.append(
    dict(zip(["PostalCode", "Borough", "Neighbourhood"], temp)), ignore_index=True)

postalData = postalData[postalData.Borough != "Not assigned"]
postalData["Neighbourhood"] = postalData["Neighbourhood"].replace({'\n':''}, regex=True)
postalData=postalData.reset_index()
postalData = postalData.drop(["index"], axis = 1)
postalData.set_value(6,'Neighbourhood', "Queen's Park")

postDataFiltered = postalData.groupby(["PostalCode"]).agg({"Borough": lambda x: select_Borough(x),
                                 "Neighbourhood": lambda x: concatenate_neighbourhood(x)})


def createPostalCodeDataFrame(url):
    postalData = pd.DataFrame(columns = ["PostalCode", "Borough", "Neighbourhood"])

    obj  = requests.get(url)

    page = obj.text

    soup = BeautifulSoup(page, "html5lib")

    #print(soup)

    dataTable = soup.find('table', class_='wikitable sortable')
    row = dataTable.find('tr')

    for row in dataTable.find_all('tr')[1:]:
        temp = []
        for cell in row.find_all('td'):
            temp.append(cell.text)
        print(temp)
        postalData = postalData.append(
        dict(zip(["PostalCode", "Borough", "Neighbourhood"], temp)), ignore_index=True)

    postalData = postalData[postalData.Borough != "Not assigned"]
    postalData["Neighbourhood"] = postalData["Neighbourhood"].replace({'\n':''}, regex=True)
    postalData=postalData.reset_index()
    postalData = postalData.drop(["index"], axis = 1)
    postalData.set_value(6,'Neighbourhood', "Queen's Park")

    postDataFiltered = postalData.groupby(["PostalCode"]).agg({"Borough": lambda x: select_Borough(x),
                                    "Neighbourhood": lambda x: concatenate_neighbourhood(x)})

['M1A', 'Not assigned', 'Not assigned\n']
['M2A', 'Not assigned', 'Not assigned\n']
['M3A', 'North York', 'Parkwoods\n']
['M4A', 'North York', 'Victoria Village\n']
['M5A', 'Downtown Toronto', 'Harbourfront\n']
['M5A', 'Downtown Toronto', 'Regent Park\n']
['M6A', 'North York', 'Lawrence Heights\n']
['M6A', 'North York', 'Lawrence Manor\n']
['M7A', "Queen's Park", 'Not assigned\n']
['M8A', 'Not assigned', 'Not assigned\n']
['M9A', 'Etobicoke', 'Islington Avenue\n']
['M1B', 'Scarborough', 'Rouge\n']
['M1B', 'Scarborough', 'Malvern\n']
['M2B', 'Not assigned', 'Not assigned\n']
['M3B', 'North York', 'Don Mills North\n']
['M4B', 'East York', 'Woodbine Gardens\n']
['M4B', 'East York', 'Parkview Hill\n']
['M5B', 'Downtown Toronto', 'Ryerson\n']
['M5B', 'Downtown Toronto', 'Garden District\n']
['M6B', 'North York', 'Glencairn\n']
['M7B', 'Not assigned', 'Not assigned\n']
['M8B', 'Not assigned', 'Not assigned\n']
['M9B', 'Etobicoke', 'Cloverdale\n']
['M9B', 'Etobicoke', 'Islington\n']
['M9B', '

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:44: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [7]:
df = pd.read_csv('TorontoPostalData.csv')
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df.shape

(103, 3)

So far for getting the data.

## Step2: enriching with Geo Coords

In this par we add longitude  and lattitude to the dataframes rows

### install the geocoder library

In [9]:

!pip install geocoder
import geocoder # import geocoder

print('\n**** installed geocoder ****\n')

     |████████████████████████████████| 102kB 17.4MB/s ta 0:00:01
     |████████████████████████████████| 829kB 27.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 17.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future

**** installed geocoder ****



### define a function for retrieving long&lattm

In [10]:

def fn_get_lattlong(postal_code):

  # initialize your variable to None
  lat_lng_coords = None

  # loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    sys.stdout.write('.')

  latitude  = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  return latitude,longitude

print('defined function')

defined function


#### Using the CSV file provided by Coursera

Prepare CSV working

In [11]:
import csv
csv_url ='https://cocl.us/Geospatial_data'

Load data into separate dataframe

In [12]:
df_csv = pd.read_csv(csv_url )
df_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### add new column and fill with default

In [17]:
df['Latitude']  = 0.0
df['Longitude'] = 0.0
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",0.0,0.0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",0.0,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0.0,0.0
3,M1G,Scarborough,Woburn,0.0,0.0
4,M1H,Scarborough,Cedarbrae,0.0,0.0


Iterate over grouped dataframe and copy long/latt from csv data



In [20]:
for index,row in df.iterrows():
    pc = row['PostalCode']
    # show PostalCode we're working on to indicate progress
    sys.stdout.write( pc + ' ' )
    
    # get coords and save to df_grouped
    latt = df_csv.loc[ df_csv['Postal Code'] == pc ]['Latitude'].values[0]
    long = df_csv.loc[ df_csv['Postal Code'] == pc ]['Longitude'].values[0]
    
    # set values in grouped df
    df.loc[ df['PostalCode'] == pc, 'Latitude' ]  = latt
    df.loc[ df['PostalCode'] == pc, 'Longitude' ] = long

M1B M1C M1E M1G M1H M1J M1K M1L M1M M1N M1P M1R M1S M1T M1V M1W M1X M2H M2J M2K M2L M2M M2N M2P M2R M3A M3B M3C M3H M3J M3K M3L M3M M3N M4A M4B M4C M4E M4G M4H M4J M4K M4L M4M M4N M4P M4R M4S M4T M4V M4W M4X M4Y M5A M5B M5C M5E M5G M5H M5J M5K M5L M5M M5N M5P M5R M5S M5T M5V M5W M5X M6A M6B M6C M6E M6G M6H M6J M6K M6L M6M M6N M6P M6R M6S M7A M7R M7Y M8V M8W M8X M8Y M8Z M9A M9B M9C M9L M9M M9N M9P M9R M9V M9W 

## Elaborated dataframe

In [21]:
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Finally, download
data for clustering in next stepm

In [22]:
## download from Watson Studio Notebook,
## see https://medium.com/ibm-data-science-experience/how-to-upload-download-files-to-from-notebook-in-my-local-machine-6a4e65a15767

import base64
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df, 'elaborated_toronto.csv', 'toronto.csv')